# serverMLX

> FASTAPI image uploader 

In [ ]:
#| default_exp serverMLX

In [1]:
#| hide
from nbdev.showdoc import *

To process the image for generating `mlx-community/Qwen2.5-VL-7B-Instruct-bf16`.<br>
As it has a upper limit to process image `1280*28*28` and image need to be saved these are few helper fuction to achive this. It is for running inference mlx. The code is not shown here can be viewed in the git repo.

In [7]:
#| export 

import sys
from finfox.core import *


prompt =  """You are an AI specialized in recognizing and extracting text from all images in markdown format.
Your mission is to analyze the image document and generate the result while maintaining data integrity.Extract the each text in details as present. 
It is mission critical. Do not send anything extra. DO NOT MISS ANY INFORMATION. Double check."""

if sys.platform == "darwin":  # init iff the code is running in mac
    from finfox.core import *
    import mlx.core as mx
    from mlx_vlm import load, generate
    from mlx_vlm.prompt_utils import apply_chat_template, get_message_json
    from mlx_vlm.utils import load_config
    # Load the model
    #m_pth = "mlx-community/Qwen2.5-VL-7B-Instruct-bf16"
    m_pth = "mlx-community/Qwen2.5-VL-7B-Instruct-8bit"
    m_pth = "mlx-community/Qwen2.5-VL-7B-Instruct-4bit"
    m_pth = "mlx-community/Qwen2.5-VL-7B-Instruct-3bit"

    model, processor = load(m_pth)
    config = load_config(m_pth)

    formatted_prompt = apply_chat_template(
            processor, config, prompt, num_images=1
        )
    
    mx.eval(model.parameters()) 

## FastAPI upload image

In [8]:
#| export
from typing import List
from PIL import Image
if sys.platform == "darwin":  

    from fastapi import FastAPI, UploadFile, File, Header, Query
    import io
    from fastcore.all import *

    app = FastAPI()

    @app.post("/upload/")
    async def upload_image(
        files: List[UploadFile] = File(...),
        N: int = Query(..., alias="N")):
        
        im_li = []
        for i, file in enumerate(files):
            # Read the image
            data = await file.read()
            im = Image.open(io.BytesIO(data))
            fn = f"temp_{N}_{i}.png"
            qwen_resize(im, limit=256*3*28*28, fn=fn)
            im_li.append(fn)
        
        if sys.platform == "darwin":  
            return generate(model, 
                            processor, 
                            formatted_prompt,
                            im_li, 
                            verbose=True, 
                            max_tokens=4_056, 
                            repetition_penalty=1.1,
                            temperature=0.2 
                        )
        
        return [len(im_li)]
    
    

In [9]:
#| export
import httpx
from typing import List
import io
    

async def test_server(images: List[Image.Image], N:int, url="http://localhost:9000/upload/"):
    """
    check uploading image to the server
    """
    try:
        files = []
        for i, img in enumerate(images):
            try:
                img_copy = img.copy()
                
                byte_arr = io.BytesIO()
                img_copy.save(byte_arr, format='PNG')
                byte_arr.seek(0)
                
                files.append(('files', (f'image_{i}.png', byte_arr.getvalue(), 'image/png')))
            except Exception as img_error:
                print(f"Error processing image {i}: {str(img_error)}")
                raise
        
        async with httpx.AsyncClient() as client:
            response = await client.post(url, 
                                         files=files, 
                                         params= {"N":str(N)},
                                         timeout=600.0)
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Server response text: {response.text}")
                return None
                
    except Exception as e:
        print(f"Error during upload: {str(e)}")
        return None

In [4]:
!ls

00_core.ipynb		03_rough.ipynb	company2.csv  nbdev.yml    styles.css
01_serverMLX.ipynb	_quarto.yml	imgs.csv      result
02_extractGemini.ipynb	company1.csv	index.ipynb   sidebar.yml


In [5]:
im1, im2 = Image.open("result/2.png"), Image.open("result/3.png")
im1.size, im2.size

((842, 1191), (842, 1191))

In [6]:
result = await test_server([im1], N=1), await test_server([im2], N=1)
assert len(result) == 2

In [7]:
import asyncio

results = await asyncio.gather(
    test_server([im1], N=1),
    test_server([im2], N=1)
)

In [9]:
show_markdown(result[1])

## About the report

Tata Motors Limited's 79th Integrated Annual Annual Report for FY24 covers our financial and non-financial performance, and provides an insight into our business model, strategy, risks, opportunities, performance, and achievements in the period under review.

### Reporting cycle
- **01 April 2023** to **31 March 2024**

### Frameworks, guidelines, and standards used
- **Integrated Reporting**: Now part of IFRS Reporting
- **International <IR> Framework**: International <IR> Framework (the <IR> Framework)
- **Ministry of Corporate Affairs**: Ministry of Corporate Affairs
- **Companies Act, 2013**: Companies Act, 2013
- **Indian Accounting Standards**: Indian Accounting Standards
- **Securities and Exchange Board of India (Listing Obligations and Disclosure Requirements) Regulations, 2015**: Securities and Exchange Board of India (Listing Obligations and Disclosure Requirements) Regulations, 2015
- **GRI Standards 2021**: GRI Standards 2021 (with reference)

### Reporting scope and boundary
Unless otherwise specified, the Report covers financial information on a consolidated basis across segments for Tata Motors Group. The operating segment comprises the automotive segment and others, with details presented for entities based on four reportable sub-segments: Tata Motors Limited, Tata Commercial Vehicles (Tata CV), Tata Passenger Vehicles (Tata PV), Jaguar Passenger Electric Mobility (JLR), and Vehicle Financing.

### Reporting boundary
The reporting boundary for non-financial data in the 'sustainability review' section covers indicators as per GRI Standards 2021 for Tata Motors Limited (CV Business), Tata Motors Passenger Vehicles Limited (TMPVL), Tata Passenger Electric Mobility Limited (TPEML), and a select set of indicators for JLR. The BRSR covers data for Tata Motors Limited on a standalone basis.

### Cautionary statements
Statements in the Integrated Report describing our objective, projections, estimates, and expectations may be forward-looking statements. Important factors that could make a difference to our operations include, among others, economic conditions affecting demand/supply and price conditions in the domestic and overseas markets, in which we operate, in addition to changes in applicable securities laws and regulations. Actual results could differ materially from those expressed or implied. Important factors that could make a difference to our stakeholders and value creation include, among others, economic conditions affecting demand/supply and price conditions in the domestic and overseas markets, in which we operate, in addition to changes in applicable securities laws and regulations, tax laws, and other statutes and incidental factors.

### Responsibility statement
Our Board ensures the integrity, completeness, and adherence to the principle of materiality to determine their inclusion in our Integrated Report. This process focuses on matters critical to stakeholders and value creation.

### Materiality
Tata Motors' top management actively reviews material topics, applying the principle of materiality to determine their inclusion in our Integrated Report. This process focuses on matters critical to stakeholders and value creation.

### Assurance
Financial statements: Audited by independent auditors BSR & Co. LLP
For non-financial data (India operations):
(a) The BRSR covers data for Tata Motors Limited on a standalone basis; a reasonable assurance on BRSR core indicators is provided by KPMG Assurance and Consulting Services LLP.
(b) Indicators covered as part of GRI Standards 2021 include data for TML, TMPVL, and TPEML; and a limited assurance on these indicators is provided by KPMG Assurance and Consulting Services LLP.

### Other details (performance measures)
EBITDA is defined to include the product development expenses charged to P&L and realised FX and realised commodity hedges, but excludes the gain/loss on realised derivatives entered into for the purpose of hedging debt, revaluation of foreign currency debt, revaluation of foreign currency assets and liabilities, MTM on FX and commodity hedges, other income (except government grant) as well as exceptional items.
EBIT is defined as reported EBITDA plus profit from equity accounted investee less depreciation and amortisation.
Auto free cash flow is defined as net cash generated from operating activities less net cash used in automotive investing activities, excluding investments in consolidated entities, M&A-linked asset purchases and movements in financial investments, and after net finance expenses and fees paid, less free cash flow of the TMF Group, i.e., the financing business.

### Reporting scope and boundary
Unless otherwise specified, the Report covers financial information on a consolidated basis across segments for Tata Motors Group. The operating segment comprises the automotive segment and others, with details presented for entities based on four reportable sub-segments: Tata Motors Limited, Tata Commercial Vehicles (Tata CV), Tata Passenger Vehicles (Tata PV), Jaguar Passenger Electric Mobility (JLR), and Vehicle Financing.

### Reporting boundary
The reporting boundary for non-financial data in the 'sustainability review' section covers indicators as per GRI Standards 2021 for Tata Motors Limited (CV Business), Tata Motors Passenger Vehicles Limited (TMPVL), Tata Passenger Electric Mobility Limited (TPEML), and a select set of indicators for JLR. The BRSR covers data for Tata Motors Limited on a standalone basis.

### Cautionary statements
Statements in the Integrated Report describing our objective, projections, estimates, and expectations may be forward-looking statements. Important factors that could make a difference to our operations include, among others, economic conditions affecting demand/supply and price conditions in the domestic and overseas markets, in which we operate, in addition to changes in applicable securities laws and regulations. Actual results could differ materially from those expressed or implied. Important factors that could make a difference to our stakeholders and value creation include, among others, economic conditions affecting demand/supply and price conditions in the domestic and overseas markets, in which we operate, in addition to changes in applicable securities laws and regulations, tax laws, and other statutes and incidental factors.

### Responsibility statement
Our Board ensures the integrity, completeness, and adherence to the principle of materiality to determine their inclusion in our Integrated Report. This process focuses on matters critical to stakeholders and value creation.

### Materiality
Tata Motors' top management actively reviews material topics, applying the principle of materiality to determine their inclusion in our Integrated Report. This process focuses on matters critical to stakeholders and value creation.

### Assurance
Financial statements: Audited by independent auditors BSR & Co. LLP
For non-financial data (India operations):
(a) The BRSR covers data for Tata Motors Limited on a standalone basis; a reasonable assurance on BRSR core indicators is provided by KPMG Assurance and Consulting Services LLP.
(b) Indicators covered as part of GRI Standards 2021 include data for TML, TMPVL, and TPEML; and a limited assurance on these indicators is provided by KPMG Assurance and Consulting Services LLP.

### Other details (performance measures)
EBITDA is defined to include the product development expenses charged to P&L and realised FX and realised commodity hedges, but excludes the gain/loss on realised derivatives entered into for the purpose of hedging debt, revaluation of foreign currency debt, revaluation of foreign currency assets and liabilities

In [10]:
import pandas as pd
df = pd.read_csv("imgs.csv")
df.head()

,fn
0,/mnt/d/finfox/ASHOKLEY/Financial Year 2010/ind...
1,/mnt/d/finfox/ASHOKLEY/Financial Year 2010/ind...
2,/mnt/d/finfox/ASHOKLEY/Financial Year 2010/ind...
3,/mnt/d/finfox/ASHOKLEY/Financial Year 2010/ind...
4,/mnt/d/finfox/ASHOKLEY/Financial Year 2010/ind...


In [11]:
#| export
import asyncio
from fastcore.utils import *

async def get_md(fn:Path, dir_name:str="QWEN_Index_md"):
    try :

        dir = fn.parent.parent/dir_name
        dir.mkdir(exist_ok=True)
        im = Image.open(fn)
        fn_ = fn.name.split('.')[0]
        md_fn = dir/f"{fn_}.md"


        # Run both test_server calls in parallel
        results = await asyncio.gather(
            test_server([im], N=1),
        )
        
        #show_markdown(str2md(results[0]))
        md_fn.write_text(results[0])
        
        print(f"{md_fn=} {len(results[0])=}")

    except Exception as e:
        print(f"error @ {fn=}  {e}")



In [12]:
for i in range(100, 500):
    print(f" {i=} ", end=" ->")
    fn = Path(df.iloc[i]['fn'])
    await get_md(fn)

 i=100  ->md_fn=Path('/mnt/d/finfox/ASHOKLEY/Financial Year 2011/QWEN_Index_md/31.md') len(results[0])=4224
 i=101  ->md_fn=Path('/mnt/d/finfox/ASHOKLEY/Financial Year 2011/QWEN_Index_md/32.md') len(results[0])=1383
 i=102  ->md_fn=Path('/mnt/d/finfox/ASHOKLEY/Financial Year 2011/QWEN_Index_md/33.md') len(results[0])=1033
 i=103  ->

In [8]:
#| hide
import aiohttp

async def test_connection():
    try:
        async with aiohttp.ClientSession() as session:
            url = "http://127.0.0.1:9000/openapi.json"
            print(f"Trying {url}")
            async with session.get(url) as response:
                print(f"Status: {response.status}")
                return await response.json()
    except Exception as e:
        print(f"Connection error details: {type(e).__name__} - {str(e)}")

#await test_connection()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()